In [4]:
from model_training import GPTConfig, train, GPT

using device: cuda


In [2]:
# CODE UPDATE HERE
max_lr = 6e-4 
min_lr = max_lr * 0.1
warmup_steps = 50
max_steps = 5000

In [6]:
gpt_config = GPTConfig()
gpt_config.B = 16

In [ ]:
print("Starting training")
train(warmup_steps=warmup_steps,
      max_steps=max_steps,
      min_lr=min_lr,
      max_lr=max_lr, 
      gpt_config=gpt_config)

In [2]:
import tiktoken
import torch
enc = tiktoken.get_encoding('gpt2') 


In [7]:
import os
model = GPT(gpt_config)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
out_dir = 'out'
model.load_state_dict(torch.load(os.path.join(out_dir, 'ckpt.pt')))
model = model.to(device)

In [8]:
num_return_sequences = 5
max_length = 30

In [10]:

text = "what services he has done for his country ?"
tokens = enc.encode(text)
tokens = torch.tensor(tokens)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
x = tokens.to('cuda')

In [11]:
from torch.nn import functional as F

def generate_output(model, input_tokens, num_return_sequences, max_length):
    x = input_tokens
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)
    while x.size(1) < max_length:
        # forward the model to get the logits
        with torch.no_grad():
            logits = model(x)[0] # (B, T, vocab_size)
            # take the logits at the last position
            logits = logits[:, -1, :] # (B, vocab_size)
            # get the probabilities
            probs = F.softmax(logits, dim=-1)
            # do top-k sampling of 50 (huggingface pipeline default)
            # topk_probs here becomes (5, 50), topk_indices is (5, 50)
            topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
            # select a token from the top-k probabilities
            # note: multinomial does not demand the input to sum to 1
            ix = torch.multinomial(topk_probs, 1) # (B, 1)
            # gather the corresponding indices
            xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
            # append to the sequence
            x = torch.cat((x, xcol), dim=1)

    output = []
    # print the generated text
    for i in range(num_return_sequences):
        tokens = x[i, :max_length].tolist()
        output.append(tokens)
        # decoded = enc.decode(tokens)

        # print(">", decoded)
    return output

In [12]:
results = generate_output(model=model, input_tokens=x,num_return_sequences=5,max_length=max_length)

In [14]:
for result in results:
    decoded = enc.decode(result)
    print(f">{decoded}")

>what services he has done for his country ? we
 the, and of you a your it and what.And will you,:
 the.
>what services he has done for his country ?!To



 and what, and- that:
What you: I with: of
>what services he has done for his country ?

;
 with me but!
 but will, to the you, to my:


>what services he has done for his country ?; a



I's.O,

 but;, all to me:

>what services he has done for his country ? you's,;

The'd is my-,?
My I; I.
I
